In [1]:
import math
from time import sleep

import numpy as np
import rerun as rr
from rerun.utilities import build_color_grid
from impostor.utils import Curve

rec = rr.new_recording("rerun_curve_notebook")

curve = Curve([(0, 0), (0.14, 0.6), (0.7, 0.7), (0.86, 0.4), (0.98, 0.1), (1, 0.0)])

rec.log("curves/parabola", rr.SeriesLine(name="blade"), timeless=True)
for t in np.linspace(0, 1, 100):
    rec.set_time_sequence("frame_nr", int(t * 100000))
    f_of_t = curve.evaluate(t)
    rec.log(
        "curves/parabola",
        rr.Scalar(f_of_t),
    )
rec.log("curves/points", rr.SeriesPoint(name="control_points"), timeless=True)
for cp in curve._control_points:
    rec.set_time_sequence("frame_nr", int(cp[0] * 100000))
    rec.log("curves/points", rr.Scalar(cp[1]))
rec.notebook_show()

Viewer()

In [2]:
from impostor.plant import Plant
import impostor.systems as syst
from impostor.utils import NormalDistribution

import rerun as rr
import time


def test_grow(iterations=120):
    plant = Plant()
    root_entity = syst.start_root(plant)
    branch_system = syst.BranchingSystem(internode_spacing=NormalDistribution(1.6, 0.1))
    relax_spring_system = syst.RelaxSpringSystem()
    secondary_growth_system = syst.SecondaryGrowthSystem()
    update_mass_above_system = syst.UpdateMassAboveSystem()

    for i in range(iterations):
        rr.set_time_sequence("frame_idx", i)
        print(f"Iteration {i}")
        syst.grow_system(plant)
        branch_system.execute(plant)    
        secondary_growth_system.execute(plant)

        if i == iterations - 1:
            syst.LeafingSystem().execute(plant)

        update_mass_above_system.execute(plant)
        relax_spring_system.execute(plant)
        syst.rr_log_components(plant)
        syst.rr_log_graph(plant)
        syst.rr_log_transforms_system(plant)
        mesh = syst.create_plant_mesh(plant)
        mesh.rr_log()

    return plant, root_entity

rr.init("impostor")
plant, root = test_grow(16)


rr.notebook_show()

Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
Iteration 10
Iteration 11
Iteration 12
Iteration 13
Iteration 14
Iteration 15
LeafMeta created with attachment_parent_entity: 9
LeafMeta created with attachment_parent_entity: 9
LeafMeta created
midrib_entities: [14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36]


/home/artefacts/repos/impostor/impostor/systems/rerun.py:100: RerunWarning: Ambiguous input for colors of length 3. If using 0xRRGGBBAA values, please wrap as np.array with dtype=np.uint32
  rr.GraphNodes(


Viewer()

In [3]:
import sys

print(sys.executable)


/home/artefacts/repos/impostor/.pixi/envs/default/bin/python3.10
